Imports

In [10]:
import feed_forward
import feed_forward_EUCL
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import geoopt
from time import time
from torchvision import datasets, transforms
torch.cuda.is_available()

True

CUDA check

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [12]:
transform = transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize((0.4915, 0.4823, 0.4468), (0.2470, 0.2435, 0.2616))
                                ])

In [13]:
trainset = datasets.CIFAR10('PATH_TO_STORE_TRAINSET', download=True, train=True, transform=transform)
valset = datasets.CIFAR10('PATH_TO_STORE_TESTSET', download=True, train=False, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [14]:
label_map = {0: 0, 2: 1}
#ball = geoopt.PoincareBall()
class_names = ['airplane', 'bird']
cifar2 = [(img, label_map[label]) for img, label in trainset if label in [0, 2]]
cifar2_val = [(img, label_map[label]) for img, label in valset if label in [0, 2]]

trainloader = torch.utils.data.DataLoader(cifar2, batch_size=512, shuffle=True)
valloader = torch.utils.data.DataLoader(cifar2_val, batch_size=512, shuffle=False)

Initializing the model

In [15]:
model = feed_forward_EUCL.EuclFF(3072, 512, 256, 2, nn.ReLU())
model

EuclFF(
  (act_fn): ReLU()
  (fc1): Linear(in_features=3072, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=2, bias=True)
)

In [16]:
learning_rate = 1e-2
momentum = 0.9

In [17]:
optimizer = optim.SGD(model.parameters(), 
                                       lr=learning_rate)
#optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
loss_fn = nn.CrossEntropyLoss()

In [20]:
n_epochs = 50
for epoch in range(n_epochs):
    for imgs, labels in trainloader:
        # img, label = img.to(device), label.to(device)
        batch_size = imgs.shape[0]
        out = model(imgs.view(batch_size, -1))
        loss = loss_fn(out, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

Epoch: 0, Loss: 0.410966
Epoch: 1, Loss: 0.407343
Epoch: 2, Loss: 0.380282
Epoch: 3, Loss: 0.415904
Epoch: 4, Loss: 0.306867
Epoch: 5, Loss: 0.403170
Epoch: 6, Loss: 0.381865
Epoch: 7, Loss: 0.384968
Epoch: 8, Loss: 0.344673
Epoch: 9, Loss: 0.352055
Epoch: 10, Loss: 0.404371
Epoch: 11, Loss: 0.366717
Epoch: 12, Loss: 0.403333
Epoch: 13, Loss: 0.351317
Epoch: 14, Loss: 0.362640
Epoch: 15, Loss: 0.343391
Epoch: 16, Loss: 0.360022
Epoch: 17, Loss: 0.368846
Epoch: 18, Loss: 0.298399
Epoch: 19, Loss: 0.359380
Epoch: 20, Loss: 0.321772
Epoch: 21, Loss: 0.332463
Epoch: 22, Loss: 0.291861
Epoch: 23, Loss: 0.332517
Epoch: 24, Loss: 0.341349
Epoch: 25, Loss: 0.269235
Epoch: 26, Loss: 0.308508
Epoch: 27, Loss: 0.298624
Epoch: 28, Loss: 0.274251
Epoch: 29, Loss: 0.275225
Epoch: 30, Loss: 0.323891
Epoch: 31, Loss: 0.299362
Epoch: 32, Loss: 0.329596
Epoch: 33, Loss: 0.241542
Epoch: 34, Loss: 0.321639
Epoch: 35, Loss: 0.258792
Epoch: 36, Loss: 0.288958
Epoch: 37, Loss: 0.273830
Epoch: 38, Loss: 0.325

In [30]:
correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in valloader:
        batch_size = imgs.shape[0]
        outputs = model(imgs.view(batch_size, -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
        
print("Accuracy: %f", correct / total)

Accuracy: %f 0.859
